In [1]:
import networkx as nx 
import pandas as pd
from geopy.distance import distance

In [2]:
metro_df = pd.read_csv('metroetrem_sp_comlinks.csv') #carregando a base de dados fornecida em um dataframe
metro_df.head(10) #visualizando como está nosso dataframe

,num_lin,nome_lin,long,lat,estacao_upp,link
0,4,AMARELA,-46.723768,-23.586440,SAO PAULO MORUMBI,BUTANTA
1,4,AMARELA,-46.708157,-23.571857,BUTANTA,PINHEIROS
2,4,AMARELA,-46.701610,-23.567392,PINHEIROS,FARIA LIMA
3,4,AMARELA,-46.694102,-23.567269,FARIA LIMA,FRADIQUE COUTINHO
4,4,AMARELA,-46.684259,-23.566228,FRADIQUE COUTINHO,OSCAR FREIRE
5,4,AMARELA,-46.671921,-23.560521,OSCAR FREIRE,PAULISTA
6,4,AMARELA,-46.662161,-23.555044,PAULISTA,HIGIENOPOLIS MACKENZIE
7,4,AMARELA,-46.652165,-23.548935,HIGIENOPOLIS MACKENZIE,REPUBLICA
8,4,AMARELA,-46.642737,-23.544291,REPUBLICA,LUZ
9,4,AMARELA,-46.634499,-23.536259,LUZ,LUZ


Vamos dividir nosso Dataframe em partes, fazendo um novo DF para cada uma das linhas de metrô. Para isso vamos visualizar quais são as linhas existentes em nosso DF

In [3]:
metro_df['nome_lin'].value_counts()

AZUL         23
DIAMANTE     22
RUBI         19
ESMERALDA    18
VERMELHA     18
LILAS        17
CORAL        16
VERDE        14
SAFIRA       13
TURQUESA     13
AMARELA      10
PRATA        10
JADE          3
Name: nome_lin, dtype: int64

Podemos ver um total de 13 linhas distintas, algumas com menos, outras com mais estações, para cada uma delas criaremos um novo DF:

In [4]:
lista_dfs = [] #inicializando uma lista vazia, onde serão salvos os dataframes separadamente
linha_azul_df = metro_df.loc[metro_df['nome_lin'] == 'AZUL'].reset_index() #cria-se o novo DF usando o método loc 
#para determinar qual a linha desejada, e o método reset_index para que os indices mantenham uma ordem condizente
lista_dfs.append(linha_azul_df) #salvamos o novo DF em nossa lista e repetimos o processo para cada linha possível
linha_prata_df = metro_df.loc[metro_df['nome_lin'] == 'PRATA'].reset_index()
lista_dfs.append(linha_prata_df)
linha_amarela_df = metro_df.loc[metro_df['nome_lin'] == 'AMARELA'].reset_index()
lista_dfs.append(linha_amarela_df)
linha_lilas_df = metro_df.loc[metro_df['nome_lin'] == 'LILAS'].reset_index()
lista_dfs.append(linha_lilas_df)
linha_verde_df = metro_df.loc[metro_df['nome_lin'] == 'VERDE'].reset_index()
lista_dfs.append(linha_verde_df)
linha_coral_df = metro_df.loc[metro_df['nome_lin'] == 'CORAL'].reset_index()
lista_dfs.append(linha_coral_df)
linha_diamante_df = metro_df.loc[metro_df['nome_lin'] == 'DIAMANTE'].reset_index()
lista_dfs.append(linha_diamante_df)
linha_safira_df = metro_df.loc[metro_df['nome_lin'] == 'SAFIRA'].reset_index()
lista_dfs.append(linha_safira_df)
linha_vermelha_df = metro_df.loc[metro_df['nome_lin'] == 'VERMELHA'].reset_index()
lista_dfs.append(linha_vermelha_df)
linha_esmeralda_df = metro_df.loc[metro_df['nome_lin'] == 'ESMERALDA'].reset_index()
lista_dfs.append(linha_esmeralda_df)
linha_jade_df = metro_df.loc[metro_df['nome_lin'] == 'JADE'].reset_index()
lista_dfs.append(linha_jade_df)
linha_rubi_df = metro_df.loc[metro_df['nome_lin'] == 'RUBI'].reset_index()
lista_dfs.append(linha_rubi_df)
linha_turquesa_df = metro_df.loc[metro_df['nome_lin'] == 'TURQUESA'].reset_index()
lista_dfs.append(linha_turquesa_df)

Vamos visualizar como ficaram estes dataframes individuais, observando como exemplo a linha Jade:

In [5]:
linha_jade_df.head()

,index,num_lin,nome_lin,long,lat,estacao_upp,link
0,89,13,JADE,-46.520822,-23.497915,ENGENHEIRO GOULART,GUARULHOS CECAP
1,90,13,JADE,-46.493596,-23.447484,GUARULHOS CECAP,AEROPORTO GUARULHOS
2,91,13,JADE,-46.493781,-23.433131,AEROPORTO GUARULHOS,AEROPORTO GUARULHOS


É de grande interesse saber a distância existente entre as duas estações que se ligam, e podemos notar que temos em cada linha as coordenadas de latitude e longitude da "estacao_upp". Podemos usar dessa informação para criar uma nova coluna em nosso DataFrame com a distancia percorrida entre as estações ligadas.
Para isso definimos a função insere_distancias, que recebe como parametro um de nossos dataframes, e para cada par de linhas consecutivas passa as informações de coordenadas para o método geopy.distance.distance, que retorna a distância geodésica entre as coordenadas (em quilometros, neste projeto). Tais distâncias são salvas em uma lista que é retornada ao final da execução.
OBS: Note que a última linha de cada DF mostra uma estação ligada a si propria, por isso ao final do loop da função adiciona-se o valor 0 na lista.

In [6]:
def insere_distancias(df):
    distancias=[]
    for i in range(df.shape[0]-1): #loop para percorrer o data frame
        #Pega as coordenadas de uma linha (estação 'atual') e da linha seguinte ('proxima' estação) e calcula a distancia entre elas
        distancia = distance((df['lat'][i], df['long'][i]), (df['lat'][i+1], df['long'][i+1])).km 
        distancias.append(round(distancia,3)) #A distancia é salva em uma lista, arredondada em 3 casas decimais
    distancias.append(0) #Ultima linha recebe 0 pois corresponderá a uma estação ligada a si mesma
    return distancias #retorna a lista com as distancias

In [7]:
grafo_metro = nx.Graph() #Vamos inicializar um grafo vazio
for df in lista_dfs: #percorrendo nossa lista de dataframes 
    df['distancias']=insere_distancias(df) #Criamos em cada um a coluna de distancias
    #Usadno o metodo from_pandas_edgelist criamos um grafo onde cada estacao é um nó ligado a uma outra estaçao e as distancias
    #entre elas são passadas como parametro para a aresta
    grafo_linha = nx.from_pandas_edgelist(df, 'estacao_upp', 'link',edge_attr='distancias') 
    #Usando o método Compose mesclamos o grafo com o que foi inicializado previamente, até que todas nossas linhas estejam
    #reunidas em um unico grafo
    grafo_metro = nx.compose(grafo_metro, grafo_linha)

Vejamos se conseguimos extrair agora diretamente do grafo a informação de distancia entre estações

In [8]:
nx.get_edge_attributes(grafo_metro, 'distancias')

{('JABAQUARA', 'CONCEICAO'): 1.21,
 ('CONCEICAO', 'SAO JUDAS'): 1.09,
 ('SAO JUDAS', 'SAUDE'): 0.807,
 ('SAUDE', 'PRACA DA ARVORE'): 0.873,
 ('PRACA DA ARVORE', 'SANTA CRUZ'): 1.295,
 ('SANTA CRUZ', 'VILA MARIANA'): 1.091,
 ('SANTA CRUZ', 'HOSPITAL SAO PAULO'): 0.88,
 ('SANTA CRUZ', 'CHACARA KLABIN'): 0.828,
 ('VILA MARIANA', 'ANA ROSA'): 0.974,
 ('ANA ROSA', 'PARAISO'): 0.667,
 ('ANA ROSA', 'CHACARA KLABIN'): 1.5,
 ('PARAISO', 'VERGUEIRO'): 0.745,
 ('PARAISO', 'BRIGADEIRO'): 0.996,
 ('VERGUEIRO', 'SAO JOAQUIM'): 0.816,
 ('SAO JOAQUIM', 'JAPAO LIBERDADE'): 0.797,
 ('JAPAO LIBERDADE', 'SE'): 0.624,
 ('SE', 'SAO BENTO'): 0.666,
 ('SE', 'ANHANGABAU'): 0.655,
 ('SE', 'PEDRO II'): 0.748,
 ('SAO BENTO', 'LUZ'): 0.818,
 ('LUZ', 'TIRADENTES'): 0.675,
 ('LUZ', 'REPUBLICA'): 1.224,
 ('LUZ', 'LUZ'): 0.0,
 ('LUZ', 'BRAS'): 2.193,
 ('LUZ', 'BARRA FUNDA'): 3.447,
 ('TIRADENTES', 'ARMENIA'): 0.672,
 ('ARMENIA', 'PORTUGUESA TIETE'): 1.107,
 ('PORTUGUESA TIETE', 'CARANDIRU'): 0.744,
 ('CARANDIRU', 'SAN

In [9]:
def melhor_caminho(origem, destino):
    trilha = nx.shortest_path(grafo_metro, source=origem, target=destino, weight='distancias', method='dijkstra')
    distancia_total = 0
    prev = trilha[0]
    
    for item in trilha[1:]:
        dist_percurso = grafo_metro[prev][item]['distancias']
        print(f'{prev} -> {item}     Distancia: {dist_percurso} Km')
        distancia_total += dist_percurso
        prev = item
    print(f'Distancia total do percurso: {round(distancia_total,3)} Km')

In [11]:
while True:
    ans = input('Informe uma estação de origem(não use acentos ou ç). Para sair digite sair\n').upper()
    if ans=='SAIR':
        break
    else:
        destino = input('Informe uma estação de destino(não use acentos ou ç).\n').upper()
    try:
        melhor_caminho(ans,destino)
    except:
        print('Dados inválidos. Tente Novamente.\n')
        

Informe uma estação de origem(não use acentos ou ç). Para sair digite sair
se
Informe uma estação de destino(não use acentos ou ç).
luz
SE -> SAO BENTO     Distancia: 0.666 Km
SAO BENTO -> LUZ     Distancia: 0.818 Km
Distancia total do percurso: 1.484 Km
Informe uma estação de origem(não use acentos ou ç). Para sair digite sair
anhangabau
Informe uma estação de destino(não use acentos ou ç).
vila matilde
ANHANGABAU -> SE     Distancia: 0.655 Km
SE -> PEDRO II     Distancia: 0.748 Km
PEDRO II -> BRAS     Distancia: 1.035 Km
BRAS -> BRESSER MOOCA     Distancia: 0.863 Km
BRESSER MOOCA -> BELEM     Distancia: 1.85 Km
BELEM -> TATUAPE     Distancia: 1.381 Km
TATUAPE -> CARRAO     Distancia: 1.285 Km
CARRAO -> PENHA     Distancia: 2.28 Km
PENHA -> VILA MATILDE     Distancia: 1.175 Km
Distancia total do percurso: 11.272 Km
Informe uma estação de origem(não use acentos ou ç). Para sair digite sair
sair
